In [31]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt

О каждой из них мы знаем:

сколько детей она родила (признак ceb) 
возраст (age)
длительность получения образования (educ)
религиозная принадлежность (religion)
идеальное, по её мнению, количество детей в семье (idlnchld)
была ли она когда-нибудь замужем (evermarr)
возраст первого замужества (agefm)
длительность получения образования мужем (heduc)
знает ли она о методах контрацепции (knowmeth)
использует ли она методы контрацепции (usemeth)
живёт ли она в городе (urban)
есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)

In [32]:
data = pd.read_csv(r'C:\Users\One\Git\COURSERA YANDEX\Course 4\week 16\задание\botswana.tsv', delimiter = '\t')
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0



# 1. Сколько разных значений принимает признак religion?

In [33]:
data.religion.describe()

count       4361
unique         4
top       spirit
freq        1841
Name: religion, dtype: object

# 2. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски?

In [34]:
data.dropna().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1834 entries, 1 to 4360
Data columns (total 15 columns):
ceb         1834 non-null int64
age         1834 non-null int64
educ        1834 non-null int64
religion    1834 non-null object
idlnchld    1834 non-null float64
knowmeth    1834 non-null float64
usemeth     1834 non-null float64
evermarr    1834 non-null int64
agefm       1834 non-null float64
heduc       1834 non-null float64
urban       1834 non-null int64
electric    1834 non-null float64
radio       1834 non-null float64
tv          1834 non-null float64
bicycle     1834 non-null float64
dtypes: float64(9), int64(5), object(1)
memory usage: 229.2+ KB


# 3. Сколько осталось пропущенных значений в признаке heduc?

In [35]:
data['nevermarr'] = data['agefm'].isna().astype(int)

In [36]:
data.drop('evermarr', axis = 1, inplace = True)
data.agefm.fillna(0, inplace = True)

In [37]:
data['heduc'][(data['nevermarr'] == 1) & (data['heduc'].isna())] = -1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [38]:
sum(data['heduc'].isna())

123

# 4. Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик ceb).

In [39]:
data['idlnchld_noans'] = 0
data['heduc_noans'] = 0
data['usemeth_noans'] = 0
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0,0,0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0,0,0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0,0,0,0


In [40]:
data['idlnchld_noans'][(data['idlnchld'].isna())] = 1
data['heduc_noans'][(data['heduc'].isna())] = 1
data['usemeth_noans'][(data['usemeth'].isna())] = 1

'''data['idlnchld_noans'][(data['nevermarr'] == 1) & (data['idlnchld'].isna())] = 1
data['heduc_noans'][(data['nevermarr'] == 1) & (data['heduc'].isna())] = 1
data['usemeth_noans'][(data['nevermarr'] == 1) & (data['usemeth'].isna())] = 1'''

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


"data['idlnchld_noans'][(data['nevermarr'] == 1) & (data['idlnchld'].isna())] = 1\ndata['heduc_noans'][(data['nevermarr'] == 1) & (data['heduc'].isna())] = 1\ndata['usemeth_noans'][(data['nevermarr'] == 1) & (data['usemeth'].isna())] = 1"

In [41]:
data['idlnchld'].fillna(-1, inplace = True)
data['heduc'].fillna(-2, inplace = True)
data['usemeth'].fillna(-1, inplace = True)

In [42]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0,0,0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0,0,0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0,0,0,0


In [43]:
data = data.dropna()

In [44]:
data.shape[0]*data.shape[1]

78264

# 5. Постройте регрессию количества детей ceb на все имеющиеся признаки методом smf.ols, как в разобранном до этого примере. Какой получился коэффициент детерминации R^2R? Округлите до трёх знаков после десятичной точки.
# 6. Обратите внимание, что для признака religion в модели автоматически создалось несколько бинарных фиктивных переменных. Сколько их?


In [45]:
import statsmodels.formula.api as smf
a = (data.columns).astype(str)

In [46]:
l = 'ceb ~ '
for i in range(1, len(data.columns)):
    l += data.columns[i]
    if i != len(data.columns)-1:
        l += ' + '

In [47]:
m1 = smf.ols(l, data=data)
fitted = m1.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Mon, 23 Dec 2019   Prob (F-statistic):               0.00
Time:                        20:50:51   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

# 7. Проверьте критерием Бройша-Пагана гомоскедастичность ошибки в построенной модели. Выполняется ли она?

Используем критерий Бройша-Пагана для проверки гомоскедастичности ошибок:

In [48]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


# 8. Удалите из модели незначимые признаки religion, radio и tv. Проверьте гомоскедастичность ошибки, при необходимости сделайте поправку Уайта.

Ошибки гетероскедастичны, значит, значимость признаков может определяться неверно. Сделаем поправку Уайта:

In [56]:
l = 'ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans'
data.drop(['religion', 'radio', 'tv'], axis = 1, inplace = True)

In [57]:
m2 = smf.ols(l, data=data)
fitted = m2.fit(cov_type='HC1')
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Mon, 23 Dec 2019   Prob (F-statistic):               0.00
Time:                        20:57:44   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

Используем критерий Бройша-Пагана для проверки гомоскедастичности ошибок:

In [58]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


Посмотрим, не стала ли модель от удаления трёх признаков значимо хуже, с помощью критерия Фишера:

In [59]:
print("F=%f, p=%f, k1=%f" % m1.fit().compare_f_test(m2.fit()))

F=0.919236, p=0.467231, k1=5.000000


In [60]:
round(0.467231, 4)

0.4672

# 9. Удалите из текущей модели usemeth_noans и usemeth. Проверьте критерием Фишера гипотезу о том, что качество модели не ухудшилось. Введите номер первой значащей цифры в достигаемом уровне значимости

In [61]:
l = 'ceb ~ age + educ + idlnchld + knowmeth + agefm + heduc + urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans'
data.drop(['usemeth_noans', 'usemeth'], axis = 1, inplace = True)

In [62]:
m3 = smf.ols(l, data=data)
fitted = m3.fit(cov_type='HC1')
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     396.4
Date:                Mon, 23 Dec 2019   Prob (F-statistic):               0.00
Time:                        21:01:30   Log-Likelihood:                -7825.7
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4335   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.1931      0.262     -4.

In [65]:
print("F=%f, p=%f, k1=%f" % m2.fit().compare_f_test(m3.fit()))

F=92.890582, p=0.000000, k1=2.000000


In [67]:
 f, p, k1 = m2.fit().compare_f_test(m3.fit())
p

3.155200948040263e-40